In [1]:
import os


In [2]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view   
import optuna
import scipy.signal as signal


In [3]:
from one.generator.univariate import UnivariateDataGenerator
from one.models import *
from one.utils import *
from one.scorer.pot import *
from one.scorer.spot import *
import jenkspy

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
plt.rcParams["figure.figsize"] = 40,10
plt.rcParams["font.size"] = 15

# Scoring Helper

In [6]:
class ScoreCounter:
    def __init__(self):
        self.tp = 0
        self.fp = 0
        self.tn = 0
        self.fn = 0
        
    def process(self, preds, labels):
        preds = preds.copy()
        labels = labels.copy()
        ground_truth_ones = np.where(labels == 1)[0]
        pred_ones = np.where(preds == 1)[0]
        
        ranges = self._consecutive(ground_truth_ones)
        
        tp, fp, tn, fn = 0, 0, 0, 0
        
        for r in ranges:
            intersect = np.intersect1d(r, pred_ones, assume_unique=True)
            if intersect.size != 0:
                tp += r.size
                preds[intersect] = 0
                pred_ones = np.where(preds == 1)[0]
            else:
                fn += r.size
            
        fp += pred_ones.size
        tn += preds.size - tp - fp - fn
        
        self.tp += tp
        self.fp += fp
        self.tn += tn
        self.fn += fn
        
        
        return
        
        
    def _consecutive(self, data, stepsize=1):
        return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)
    
    
    @property
    def tpr(self):
        return self.tp/(self.fn+self.tp)
    
    @property
    def fpr(self):
        return self.fp/(self.tn+self.fp)
    
    @property
    def tnr(self):
        return self.tn/(self.tn+self.fp)
        
    @property
    def fnr(self):
        return self.fn/(self.fn+self.tp)
        
    @property
    def precision(self):
        return self.tp/(self.tp+self.fp)
    
    @property
    def recall(self):
        return self.tp/(self.tp+self.fn)
    
    @property
    def f1(self):
        if self.precision + self.recall == 0: return 0
        return (2*self.precision*self.recall)/(self.precision+self.recall)
    
    

## Scores

### AUMVC

In [7]:
"""
UNSUPERVISED LEARNING METRICS
Author:     Bob Stienen
License:    MIT License
Source:     http://www.github.com/bstienen/AUMVC

Implementation of the Area under the Mass-Volume Curve algorithm as by
- Stephan Clémençon and Jeremie Jakubowicz, Scoring anomalies: a M-estimation
  formulation approach. 2013-04

Implementation is inspired by
   https://github.com/albertcthomas/anomaly_tuning
"""

import warnings
import numpy as np
from scipy.special import comb
from sklearn.metrics import auc


def aumvc(scoring_function,
          X_test,
          N_mc=100000,
          N_levelsets=100,
          normalise=True):
    """ Calculate the area under the mass-volume curve for an anomaly detection
    function or algorithm

    This function uses monte carlo sampling in the parameter space box spanned
    by the provided test data in order to estimate the level set of the
    scoring function. For higher dimensionalities the amount of sampled data
    points would yield this algorithm intractable. In these cases the use of
    the `aumvc_hd` function is advised instead.

    Parameters
    ----------
    scoring_function: function
        Function that takes datapoints as numpy.ndarray (nPoints, nFeatures)
        and returns an anomaly score. This score should be in range [0,1],
        where 1 indicates the point not being an anomaly (and 0 that the point
        *is* an anomaly).
    X_test: numpy.ndarray of shape (nPoints, nFeatures)
        Datapoints used for testing the algorithm.
    N_mc: int (default: 100,000)
        Number of datapoints to sample in the parameter space to estimate the
        level sets of the scoring function.
    N_levelsets: int (default: 100)
        Number of level sets to evaluate.
    normalise: bool (default: True)
        Indicates if output scores of the scoring_function should be normalised
        before calculating the mass-volume curve. """

    # Get ranges for the test data
    mins = np.amin(X_test, axis=0)
    maxs = np.amax(X_test, axis=0)
    
    if X_test.ndim==1:
        mins = np.array([mins])
        maxs = np.array([maxs])
       

    # Generate uniform MC data
    U = np.random.rand(N_mc, len(mins))*(maxs-mins)+mins

    # Calculate volume of total cube
    vol_tot_cube = np.prod(maxs - mins)

    # Score test and MC data
    score_U = scoring_function(U)
    score_test = scoring_function(X_test)

    # Do normalising if needed
    if normalise:
        minimum = min(np.amin(score_U), np.amin(score_test))
        maximum = max(np.amax(score_U), np.amax(score_test))
        score_U = (score_U - minimum) / (maximum - minimum)
        score_test = (score_test - minimum) / (maximum - minimum)

    # Calculate alphas to use
    alphas = np.linspace(0, 1, N_levelsets)

    # Compute offsets
    offsets = np.percentile(score_test, 100 * (1 - alphas))

    # Compute volumes of associated level sets
    volumes = (np.array([np.mean(score_U >= offset)
                        for offset in offsets]) * vol_tot_cube)

    # Calculating area under the curve
    area = auc(alphas, volumes)

    # Return area and curve variables
    return (area, alphas, volumes)

In [1179]:
print("test")

test


### Label-Based Metrics

In [8]:
def qualitative_metrics(data, preds, window=10):
    """
    Returns a few quantitative metrics for us to use for evaluation when labels
    are not provided.
    Parameters
    -----------
    df: pd.Dataframe
        the dataframe with 'timestamp', 'value' and 'predict' columns
        where 'predict' is 1 for those predicted as anomalous and 0 otherwise.
    Returns
    ----------
    tuple
        (number of anomalies,
        % of anomnalies,
        avg. distance between mean and all anomalies (yaxis),
        avg. time distance between consecutive anomalies,
        avg. cycle distance between consecutive anomalies,
        maximum range between non anomaly points (yaxis)
        )
    """
    num_anomalies = preds.sum()
    percent_anomalies = num_anomalies/len(preds)

    mean_val = data.mean(axis=0)

    pred_anomalies = data[preds == 1]
    pred_non_anomalies = data[preds == 0]

    avg_anom_dist_from_mean_val = np.linalg.norm(pred_anomalies - mean_val, axis=-1).mean()
    
    
    if 1 < (preds==1).sum() < len(data):
        avg_cycles_delta_between_anomalies = np.diff(np.where(preds==1)[0]).mean()
    else: 
        avg_cycles_delta_between_anomalies = 0
        
    if 0 < (preds==1).sum() < len(data):
        max_range_non_anomalies = (np.abs(pred_non_anomalies.max() - pred_non_anomalies.min())).mean()
    else: 
        max_range_non_anomalies = 1e5
 
     
    
    # make sure window is odd
    if window % 2 == 0: window += 1
    padding = window//2
    
    # local difference filter
    fil = window
    fil = np.full((window),-1/(window-1))
    fil[padding] = 1
    if data.ndim > 1:
        fil = np.tile(fil, (data.shape[1], 1)).T
 
        
    # maximize
    # abs-trend avg
    grads = signal.savgol_filter(data, window, 1, deriv=1, axis=0)
    grads = np.abs(grads)
    conv = np.abs(signal.convolve(grads, fil, mode="valid"))
    conv = np.pad(conv, (padding, padding), mode="edge")
    diff_mean_trend = (conv[preds==1].mean(axis=0) - conv[preds==0].mean(axis=0)).sum() if 0<(preds == 1).sum()<preds.size else 0
    
    # maximize
    # ~= midpoint minus avg. of sides
    conv = np.abs(signal.convolve(data, fil, mode="valid"))
    conv = np.pad(conv, (padding, padding), mode="edge")
    diff_mid_avg = (conv[preds==1].mean(axis=0) - conv[preds==0].mean(axis=0)).sum() if 0<(preds == 1).sum()<preds.size else 0

    return (
            percent_anomalies, # min
            avg_anom_dist_from_mean_val, #max
            avg_cycles_delta_between_anomalies, #max
            max_range_non_anomalies, #max
            diff_mean_trend, #max
            diff_mid_avg #max
           )


In [919]:
data = tune_data
preds = preds

window=10
if window % 2 == 0: window += 1
padding = window//2
 
fil = window
fil = np.full((window),-1/(window-1))
fil[padding] = 1
if data.ndim > 1:
    fil = np.tile(fil, (data.shape[1], 1)).T

grads = signal.savgol_filter(data, window, 1, deriv=1, axis=0)
grads = np.abs(grads)
conv = np.abs(signal.convolve(grads, fil, mode="valid"))
conv = np.pad(conv, (padding, padding), mode="edge")
diff_mean_trend = (conv[preds==1].mean(axis=0) - conv[preds==0].mean(axis=0)).sum() if 0<(preds == 1).sum()<preds.size else 0


# Run Experiments

## F1 Tuned - 50% dataset

### -- Setup

In [9]:
from one.models import *
from one.utils import *
from one.scorer.pot import *
from numpy.lib.stride_tricks import sliding_window_view   

In [10]:
PATH0 = "../data/univar-synth/point_global/"
PATH1 = "../data/univar-synth/point_contextual/"
PATH2 = "../data/univar-synth/collective_global/"
PATH3 = "../data/univar-synth/collective_trend/"
PATH4 = "../data/univar-synth/collective_seasonal/"

In [11]:
PATHS = [PATH0, PATH1, PATH2, PATH3, PATH4]

In [12]:
SAVE_DIR = "../results/univar-synth/unsupervised-metrics/"

In [13]:
optuna.logging.set_verbosity(optuna.logging.CRITICAL)

### MA

#### Baseline

In [1160]:
# MA Model 
import warnings
warnings.filterwarnings('ignore')

"""
Qualitative Metrics:
---------------------------------------
    avg_anom_dist_from_mean_val
    avg_cycles_delta_between_anomalies
    max_range_non_anomalies
    diff_mean_trend
    midpoint_avg_difference
"""


idx_dict = {
    "avg_anom_dist_from_mean_val": 1,
    "avg_cycles_delta_between_anomalies": 2,
    "max_range_non_anomalies": 3,
    "max_range_non_anomalies-min": 3,
    "diff_mean_trend": 4,
    "diff_mid_avg": 5,

}

metric_name = "aumvc-max_range_non_anomalies-min"


for path in PATHS:
    file_list = ["-".join(f.split("-")[:-1]) for f in get_files_from_path(path) if "train" in f]
    scorer = ScoreCounter()
    for f in file_list:
        train = np.loadtxt(path+f+"-train.txt")
        test = np.loadtxt(path+f+"-test.txt")
        labels = np.loadtxt(path+f+"-labels.txt")

        n_tune = labels.size // 2
        tune_data, tune_labels = test[:n_tune], labels[:n_tune]
        test_data, test_labels = test[n_tune:], labels[n_tune:]
        
        def objective(trial):
            window = trial.suggest_int("window", 10, 150)
            contam = trial.suggest_float("contam", 0.90, 0.999)
            q = trial.suggest_float("q", 1e-5, 1e-1, log=True)
            
            test_extend = np.concatenate((train[-window:], tune_data))
            model = MovingAverageModel(window)
            scores = np.abs(model.get_scores(test_extend)[window:])
            
            aumvc_metric = aumvc(lambda x: np.abs(model.get_scores(x.flatten()))[window:], tune_data)[0] if "aumvc" in metric_name else 0
            
            #get threshold
            thres = pot(scores, q, contam) # pot
            
            preds = scores.copy()
            preds[preds <= thres] = 0
            preds[preds > thres] = 1
            
            metric = qualitative_metrics(tune_data, preds, window)
            return metric[idx_dict[metric_name.replace("aumvc-", "")]], aumvc_metric
        
         
        study = optuna.create_study(directions=["minimize", "minimize"])
        study.optimize(objective, n_trials=200)
       
        window = study.best_trials[0].params["window"]
        contam = study.best_trials[0].params["contam"]
        q = study.best_trials[0].params["q"]
        
        model = MovingAverageModel(window)
        
        test_extend = np.concatenate((train[-window:], test))
        scores = np.abs(model.get_scores(test_extend)[window:] )
        
        #get threshold
        thres = pot(scores[:n_tune], q, contam) # pot

        preds = scores[n_tune:].copy()
        preds[preds <= thres] = 0
        preds[preds > thres] = 1

        scorer.process(preds, labels[n_tune:])
        
        # Save results
        save = SAVE_DIR+f"ma/{metric_name}/"+f
        os.makedirs(SAVE_DIR+f"ma/{metric_name}", exist_ok=True)
        params = study.best_trials[0].params
        params.update({"thres": thres})
        np.savetxt(save+"-scores.txt", scores[n_tune:], header=str(params))
        np.savetxt(save+"-preds.txt", preds, header=str(params))


    print(f"{scorer.tp}, {scorer.fp}, {scorer.tn}, {scorer.fn}, {scorer.tpr}, {scorer.fpr}, {scorer.tnr}, {scorer.fnr}, {scorer.precision}, {scorer.recall}, {scorer.f1}")

285, 389, 9121, 205, 0.5816326530612245, 0.040904311251314406, 0.9590956887486856, 0.41836734693877553, 0.4228486646884273, 0.5816326530612245, 0.4896907216494846
48, 1295, 8231, 426, 0.10126582278481013, 0.13594373294142348, 0.8640562670585765, 0.8987341772151899, 0.035740878629932984, 0.10126582278481013, 0.05283434232250963
200, 414, 9086, 300, 0.4, 0.04357894736842105, 0.956421052631579, 0.6, 0.3257328990228013, 0.4, 0.3590664272890485
100, 2487, 7313, 100, 0.5, 0.2537755102040816, 0.7462244897959184, 0.5, 0.038654812524159254, 0.5, 0.07176175098672408
767, 489, 8744, 0, 1.0, 0.05296220080147298, 0.947037799198527, 0.0, 0.6106687898089171, 1.0, 0.7582797825012357


#### Combo

In [ ]:
# MA Model 
import warnings
warnings.filterwarnings('ignore')

"""
Qualitative Metrics:
---------------------------------------
    avg_anom_dist_from_mean_val
    avg_cycles_delta_between_anomalies
    max_range_non_anomalies
    diff_mean_trend
    midpoint_avg_difference
"""


idx_dict = {
    "avg_anom_dist_from_mean_val": 1,
    "avg_cycles_delta_between_anomalies": 2,
    "max_range_non_anomalies-min": 3,
    "diff_mean_trend": 4,
    "diff_mid_avg": 5,

}

metric_name = "combo-01234-min_max_max_min_max"


for path in PATHS:
    file_list = ["-".join(f.split("-")[:-1]) for f in get_files_from_path(path) if "train" in f]
    scorer = ScoreCounter()
    for f in file_list:
        train = np.loadtxt(path+f+"-train.txt")
        test = np.loadtxt(path+f+"-test.txt")
        labels = np.loadtxt(path+f+"-labels.txt")

        n_tune = labels.size // 2
        tune_data, tune_labels = test[:n_tune], labels[:n_tune]
        test_data, test_labels = test[n_tune:], labels[n_tune:]
        
        def objective(trial):
            window = trial.suggest_int("window", 10, 150)
            contam = trial.suggest_float("contam", 0.90, 0.999)
            q = trial.suggest_float("q", 1e-5, 1e-1, log=True)
            
            test_extend = np.concatenate((train[-window:], tune_data))
            model = MovingAverageModel(window)
            scores = np.abs(model.get_scores(test_extend)[window:])
            
            #aumvc_metric = aumvc(lambda x: np.abs(model.get_scores(x.flatten()))[window:], tune_data)[0] if "aumvc" in metric_name else 0
            
            #get threshold
            thres = pot(scores, q, contam) # pot
            
            preds = scores.copy()
            preds[preds <= thres] = 0
            preds[preds > thres] = 1
            
            metric = qualitative_metrics(tune_data, preds, window)
            return metric[0], metric[1], metric[2], metric[3], metric[4]
        
         
        study = optuna.create_study(directions=["minimize", "maximize", "maximize", "minimize",  "maximize"])
        study.optimize(objective, n_trials=200)
       
        window = study.best_trials[0].params["window"]
        contam = study.best_trials[0].params["contam"]
        q = study.best_trials[0].params["q"]
        
        model = MovingAverageModel(window)
        
        test_extend = np.concatenate((train[-window:], test))
        scores = np.abs(model.get_scores(test_extend)[window:] )
        
        #get threshold
        thres = pot(scores[:n_tune], q, contam) # pot

        preds = scores[n_tune:].copy()
        preds[preds <= thres] = 0
        preds[preds > thres] = 1

        scorer.process(preds, labels[n_tune:])
        
        # Save results
        save = SAVE_DIR+f"ma/{metric_name}/"+f
        os.makedirs(SAVE_DIR+f"ma/{metric_name}", exist_ok=True)
        params = study.best_trials[0].params
        params.update({"thres": thres})
        np.savetxt(save+"-scores.txt", scores[n_tune:], header=str(params))
        np.savetxt(save+"-preds.txt", preds, header=str(params))


    print(f"{scorer.tp}, {scorer.fp}, {scorer.tn}, {scorer.fn}, {scorer.tpr}, {scorer.fpr}, {scorer.tnr}, {scorer.fnr}, {scorer.precision}, {scorer.recall}, {scorer.f1}")

In [ ]:
%%capture
# NBEATS Model 


import warnings
warnings.filterwarnings('ignore')

"""
Qualitative Metrics:
---------------------------------------
    avg_anom_dist_from_mean_val
    avg_cycles_delta_between_anomalies
    max_range_non_anomalies
    diff_mean_trend
    midpoint_avg_difference
"""


idx_dict = {
    "avg_anom_dist_from_mean_val": 1,
    "avg_cycles_delta_between_anomalies": 2,
    "max_range_non_anomalies-min": 3,
    "diff_mean_trend": 4,
    "diff_mid_avg": 5,

}

metric_name = "combo-1234-max_max_min_max"


for path in PATHS:
    file_list = ["-".join(f.split("-")[:-1]) for f in get_files_from_path(path) if "train" in f]
    scorer = ScoreCounter()
    for f in file_list:
        train = np.loadtxt(path+f+"-train.txt")
        test = np.loadtxt(path+f+"-test.txt")
        labels = np.loadtxt(path+f+"-labels.txt")

        n_tune = labels.size // 2
        tune_data, tune_labels = test[:n_tune], labels[:n_tune]
        test_data, test_labels = test[n_tune:], labels[n_tune:]
        
        def objective(trial):
            window = trial.suggest_int("window", 10, 150)
            n_steps = trial.suggest_int("n_steps", 1, 5, log=True)
            contam = trial.suggest_float("contam", 0.90, 0.999)
            q = trial.suggest_float("q", 1e-5, 1e-1, log=True)
            
            test_extend = np.concatenate((train[-window:], tune_data))
            model = NBEATSModel(window, n_steps, use_gpu=True)
            model.fit(train)
            scores = model.get_scores(test_extend)[0]
            
            
            #get threshold
            thres = pot(scores, q, contam) # pot
            
            preds = scores.copy()
            preds[preds <= thres] = 0
            preds[preds > thres] = 1
            
            metric = qualitative_metrics(tune_data, preds, window)
            return  metric[1], metric[2], metric[3], metric[4]
        
         
        study = optuna.create_study(directions=["maximize", "maximize", "minimize", "maximize"])
        study.optimize(objective, n_trials=35)
       
        window = study.best_trials[0].params["window"]
        n_steps = study.best_trials[0].params["n_steps"]
        contam = study.best_trials[0].params["contam"]
        q = study.best_trials[0].params["q"]
        
        model = NBEATSModel(window, n_steps, use_gpu=True)
        model.fit(train)
        
        test_extend = np.concatenate((train[-window:], test))
        scores = model.get_scores(test_extend)[0]
        
        #get threshold
        thres = pot(scores[:n_tune], q, contam) # pot

        preds = scores[n_tune:].copy()
        preds[preds <= thres] = 0
        preds[preds > thres] = 1

        scorer.process(preds, labels[n_tune:])
        
        # Save results
        save = SAVE_DIR+f"nbeats/{metric_name}/"+f
        os.makedirs(SAVE_DIR+f"nbeats/{metric_name}", exist_ok=True)
        params = study.best_trials[0].params
        params.update({"thres": thres})
        np.savetxt(save+"-scores.txt", scores[n_tune:], header=str(params))
        np.savetxt(save+"-preds.txt", preds, header=str(params))


    print(f"{scorer.tp}, {scorer.fp}, {scorer.tn}, {scorer.fn}, {scorer.tpr}, {scorer.fpr}, {scorer.tnr}, {scorer.fnr}, {scorer.precision}, {scorer.recall}, {scorer.f1}")

2022-08-10 20:44:40 pytorch_lightning.accelerators.gpu INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[2022-08-10 20:44:42,501] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 758 samples.
[2022-08-10 20:44:42,501] INFO | darts.models.forecasting.torch_forecasting_model | Train dataset contains 758 samples.
2022-08-10 20:44:42 darts.models.forecasting.torch_forecasting_model INFO: Train dataset contains 758 samples.
[2022-08-10 20:44:42,591] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 32-bits; casting model to float32.
[2022-08-10 20:44:42,591] INFO | darts.models.forecasting.torch_forecasting_model | Time series values are 32-bits; casting model to float32.
2022-08-10 20:44:42 darts.models.forecasting.torch_forecasting_model INFO: Time series values are 32-bits; casting model to float32.
2022-08-10 20:44:42 pytorch_lightning.utilities.rank_zero INFO: GPU available: True, used: True
2022-08-10 20:44:42 pytorch_light

## Load Results

In [ ]:
SAVE_DIR = "../results/univar-synth/unsupervised-metrics/"
model = "nbeats"
metric = "combo-1234-max_max_min_max"
for path in PATHS:
    scorer = ScoreCounter()
    file_list = ["-".join(f.split("-")[:-1]) for f in get_files_from_path(path) if "train" in f]
    for f in file_list:
        preds = np.loadtxt(SAVE_DIR+f"{model}/{metric}/"+f+"-preds.txt")
        labels = np.loadtxt(path+f+"-labels.txt")[-len(preds):]
        scorer.process(preds, labels)
        
    print(f"{scorer.tp}, {scorer.fp}, {scorer.tn}, {scorer.fn}, {scorer.tpr}, {scorer.fpr}, {scorer.tnr}, {scorer.fnr}, {scorer.precision}, {scorer.recall}, {scorer.f1}")